In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [25]:
input_cols = ['team_one_color', 'team_one_name', 'team_one_player1',
       'team_one_player2', 'team_one_player3', 'team_one_player4',
       'team_one_player5', 'team_two_color', 'team_two_name',
       'team_two_player6', 'team_two_player7', 'team_two_player8',
       'team_two_player9', 'team_two_player10', 'text',
       'video_link']

output_cols = ['output']


In [38]:
df = pd.read_csv('data/lol_data.csv')
df['output'] = df['win_team']==df['team_one_name']


X_train, X_test, y_train, y_test = train_test_split(df['text'], df['output'], test_size=0.33, random_state=42)

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df['text'])
X_train_counts.shape

(10, 4388)

In [21]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape


(10, 4388)

In [30]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, df[output_cols].values.ravel())

In [42]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None))])
text_clf.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [43]:
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.5